Code to run SOCNN model on sample toy dataset.

In [1]:
import nnts
from nnts.utils import *

Working directory: 'C:\\Users\\mbinkowski\\cdsol-r-d.cluster\\cdsol-r-d.machine_learning_studies\\nntimeseries\\'


Using TensorFlow backend.


Defining sample data frame:
- column A enumerates entries, B contains random binomial variables, 
- columns C and D contain random noise
- column E is a sum of last 10 values of B multiplied by D

In [2]:
df = pd.DataFrame({'A': np.arange(1000), 
                   'B': (np.random.rand(1000)> .5) * 1.0, 
                   'C': np.random.rand(1000), 
                   'D': np.random.rand(1000)})
df['E'] = df['B'] * df['D'] 
df['E'] = np.cumsum(df['E'])
df.loc[10:, 'E'] -= np.array(df['E'][:-10])
print(df.head(20))

     A    B         C         D         E
0    0  0.0  0.885082  0.966348  0.000000
1    1  1.0  0.308109  0.982872  0.982872
2    2  0.0  0.396903  0.577059  0.982872
3    3  0.0  0.253507  0.463121  0.982872
4    4  0.0  0.410421  0.910007  0.982872
5    5  1.0  0.707546  0.535014  1.517886
6    6  0.0  0.134755  0.147408  1.517886
7    7  0.0  0.837163  0.550011  1.517886
8    8  1.0  0.155122  0.682505  2.200391
9    9  1.0  0.694945  0.911017  3.111408
10  10  1.0  0.789627  0.182675  3.294083
11  11  0.0  0.852493  0.047687  2.311210
12  12  0.0  0.410690  0.968742  2.311210
13  13  0.0  0.585119  0.057383  2.311210
14  14  0.0  0.258563  0.801456  2.311210
15  15  1.0  0.808278  0.774706  2.550903
16  16  1.0  0.568558  0.589212  3.140115
17  17  0.0  0.627001  0.256232  3.140115
18  18  0.0  0.464990  0.236566  2.457610
19  19  1.0  0.064109  0.630518  2.177111


Saving data frame to csv. Note that csv format is required by the model to read from.

In [3]:
dataset_file = os.path.join(WDIR, 'data', 'example1.csv')
df.to_csv(dataset_file)

Speifying parameters for training. For each of the keyword parameters we define a list of values for grid search.

We want to train models that predict column A given B, C and D, and A and E, given B, C and D.  

In [4]:
param_dict = dict(
    # input parameters
    input_column_names = [['B', 'C', 'D']],            # input columns 
    target_column_names = [['E'], ['A', 'E']],         # target columns
    diff_column_names = [[]],                          # columns to take first difference of   
    verbose = [2],                  # verbosity
    train_share = [(.8, .9, 1.)],   # delimeters of the training and validation shares
    input_length = [20],            # input length (1 - stateful lstm)
    output_length = [1],            # no. of timesteps to predict (only 1 impelemented)
    batch_size = [16],              # batch size
    objective=['regr'],             # only 'regr' (regression) implemented
    #training_parameters
    patience = [5],                 # no. of epoch after which learning rate will decrease if no improvement
    reduce_nb = [2],                # no. of learning rate reductions
    lr = [.01],                     # initial learning rate
    clipnorm = [1.0],               # max gradient norm
    #model parameters
    norm = [10],                    # max norm for fully connected top layer's weights
    filters = [8],                  # no. of convolutional filters per layer
    act = ['leakyrelu'],            # activation ('linear', 'relu', 'sigmoid', 'tanh', 'leakyrelu')
    kernelsize = [[1, 3], 1, 3],    # kernel size (if list of ints passed kernel size changes successively in consecutive layers)
    layers_no = [{'sigs': 5, 'offs': 1}],                # no. of layers for significance and offset sub-networks             
    architecture = [{'softmax': True, 'lambda': False}], # final activation: lambda=True indicates softplus   
    nonnegative = [False],          # if True, apply only nonnegative weights at the top layer
    connection_freq = [2],          # vertical connection frequency for ResNet
    aux_weight = [0.1],             # auxilllary loss weight
    shared_final_weights = [False], # if True, same weights of timesteps for all dimentions are trained
    resnet = [False],               # if True, adds vertical connections
)

Model class import. Other benchmark models such as LSTM, CNN or ResNet can be found in nnts.models module.

In [5]:
from nnts.models import SOCNN

Running the grid seach. Results will be pickled in 'nnts/results' directory.

In [6]:
save_file = os.path.join(WDIR, 'results', 'example_model.pkl')
runner = nnts.utils.ModelRunner(param_dict, [dataset_file], save_file)

results = runner.run(SOCNN.SOCNNmodel, log=False, limit=1)

Found 0 (< limit = 1) computed results for the setting.
As yet, for this configuration: success: 0, errors: 0
shared_final_weights: False
   input_length: 20
diff_column_names: []
    train_share: (0.8, 0.9, 1.0)
        verbose: 2
      objective: regr
      layers_no: {'offs': 1, 'sigs': 5}
     aux_weight: 0.1
   architecture: {'softmax': True, 'lambda': False}
       patience: 5
      reduce_nb: 2
         resnet: False
        filters: 8
target_column_names: ['E']
    nonnegative: False
             lr: 0.01
input_column_names: ['B', 'C', 'D']
     kernelsize: [1, 3]
connection_freq: 2
            act: leakyrelu
     batch_size: 16
       clipnorm: 1.0
           norm: 10
  output_length: 1
using <class 'nnts.models.SOCNN.SOCNNmodel'> to build the model
Total model parameters: 606
INFO:tensorflow:Summary name significance1/kernel:0 is illegal; using significance1/kernel_0 instead.
INFO:tensorflow:Summary name significance1/kernel:0 is illegal; using significance1/kernel_0 instead.

Results are stored in a list of dictionaries. Each dictionary stores parameters and results (loss function values troughout consecutive epochs) that correspond to each single model fitted during training.

In [7]:
for i, r in enumerate(results[:3]):
    print('########## results[%d] ##########' % i)
    for k, v in r.items():
        print(str(k).rjust(25) + ': ' + str(v)[:80] + ' ...' * (len(str(v)) > 80 ))
    print('\n\n')

########## results[0] ##########
                 val_loss: [1.200828766822815, 1.1899645805358887, 1.1834513425827027, 1.2146254062652588,  ...
                     hdf5: C:\Users\mbinkowski\cdsol-r-d.cluster\cdsol-r-d.machine_learning_studies\nntimes ...
     val_main_output_loss: [1.0031337857246398, 0.99870891571044917, 1.0176213026046752, 1.0649897098541259 ...
        diff_column_names: []
               kernelsize: [1, 3]
         main_output_loss: [0.99644049008687341, 0.98037807022531831, 0.96175576373934746, 0.96176830058296 ...
                     loss: [1.2353702460726101, 1.1772585660219193, 1.1386360277732213, 1.1081571330626805, ...
                layers_no: {'offs': 1, 'sigs': 5}
                     data: C:\Users\mbinkowski\cdsol-r-d.cluster\cdsol-r-d.machine_learning_studies\nntimes ...
             architecture: {'softmax': True, 'lambda': False}
                 patience: 5
             total_params: 606.0
                reduce_nb: 2
                   resnet: F